# CRQ1

#### Does the 'fare per mile' change among boroughs? aka, which boroughs charge the passengers with more money per mile?
#### We need to consider the traffic congestion too. The price might depend on the trafic taxis find in their way. 

In [6]:
import numpy as np
import pandas as pd
from datetime import datetime
import time

%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv(r'C:\Users\samin\Desktop\Homework2\Data\yellow_tripdata_2018-01.csv', usecols =['tpep_pickup_datetime','trip_distance','PULocationID','fare_amount'])

In [3]:
df.head()

,tpep_pickup_datetime,trip_distance,PULocationID,fare_amount
0,2018-01-01 00:21:05,0.5,41,4.5
1,2018-01-01 00:44:55,2.7,239,14.0
2,2018-01-01 00:08:26,0.8,262,6.0
3,2018-01-01 00:20:22,10.2,140,33.5
4,2018-01-01 00:09:18,2.5,246,12.5


In [5]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759874 entries, 0 to 8759873
Data columns (total 4 columns):
tpep_pickup_datetime    object
trip_distance           float64
PULocationID            int64
fare_amount             float64
dtypes: float64(2), int64(1), object(1)
memory usage: 835.4 MB


In [12]:
#Change data type to reduce memory usage
df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'])

df['trip_distance']=df['trip_distance'].astype(np.float16)
df['PULocationID']=df['PULocationID'].astype(np.uint16)
df['fare_amount']=df['fare_amount'].astype(np.float16)

In [13]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759874 entries, 0 to 8759873
Data columns (total 4 columns):
tpep_pickup_datetime    datetime64[ns]
trip_distance           float16
PULocationID            uint16
fare_amount             float16
dtypes: datetime64[ns](1), float16(2), uint16(1)
memory usage: 117.0 MB


In [14]:
#We need to extract Month from out datetime object
df['Month'] = pd.DatetimeIndex(df['tpep_pickup_datetime']).month

In [15]:
df.head()

,tpep_pickup_datetime,trip_distance,PULocationID,fare_amount,Month
0,2018-01-01 00:21:05,0.500000,41,4.5,1
1,2018-01-01 00:44:55,2.699219,239,14.0,1
2,2018-01-01 00:08:26,0.799805,262,6.0,1
3,2018-01-01 00:20:22,10.203125,140,33.5,1
4,2018-01-01 00:09:18,2.500000,246,12.5,1


In [17]:
df.drop(columns=['tpep_pickup_datetime'], inplace=True)

In [19]:
df.info(memory_usage ='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759874 entries, 0 to 8759873
Data columns (total 4 columns):
trip_distance    float16
PULocationID     uint16
fare_amount      float16
Month            int64
dtypes: float16(2), int64(1), uint16(1)
memory usage: 117.0 MB


In [20]:
df.head()

,trip_distance,PULocationID,fare_amount,Month
0,0.500000,41,4.5,1
1,2.699219,239,14.0,1
2,0.799805,262,6.0,1
3,10.203125,140,33.5,1
4,2.500000,246,12.5,1


In [21]:
df.groupby('Month').count()

,trip_distance,PULocationID,fare_amount
Month,,,
1,8759557,8759557,8759557
2,71,71,71
7,6,6,6
12,240,240,240


In [23]:
#delete those rows with month!=1
df.drop(df[(df.Month == 2) | (df.Month == 7) | (df.Month == 12)].index, inplace=True)

In [24]:
df.info(memory_usage ='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8759557 entries, 0 to 8759873
Data columns (total 4 columns):
trip_distance    float16
PULocationID     uint16
fare_amount      float16
Month            int64
dtypes: float16(2), int64(1), uint16(1)
memory usage: 183.8 MB


In [25]:
df['Month']=df['Month'].astype(np.uint16)

In [26]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8759557 entries, 0 to 8759873
Data columns (total 4 columns):
trip_distance    float16
PULocationID     uint16
fare_amount      float16
Month            uint16
dtypes: float16(2), uint16(2)
memory usage: 133.7 MB


In [27]:
df.head()

,trip_distance,PULocationID,fare_amount,Month
0,0.500000,41,4.5,1
1,2.699219,239,14.0,1
2,0.799805,262,6.0,1
3,10.203125,140,33.5,1
4,2.500000,246,12.5,1


In [ ]:
#clean based on two other columns: 'trip_distance' and  'fare_amount'
#fare_amount shouldn't be zero also trip_distance. Negative values also make the dataset dirty so we get rid of them. 

df['trip_distance'].plot.bar()